In [ ]:
#pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 2.3 MB/s 


In [ ]:
############################ For regression: f_regression, mutual_info_regression
############################ For classification: chi2, f_classif, mutual_info_classif
from sklearn.neighbors import NearestNeighbors,KNeighborsClassifier, KNeighborsRegressor, RadiusNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, confusion_matrix, accuracy_score, classification_report, mean_squared_error
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import category_encoders as ce
from imblearn.over_sampling import SMOTE, ADASYN
%matplotlib inline
from sklearn.linear_model import RidgeCV
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_regression, mutual_info_regression, mutual_info_classif, chi2
from time import time
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/sample_data/cancer.csv')

In [ ]:
df.head(5)

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
0,842302,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,M
1,842517,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,M
2,84300903,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,M
3,84348301,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,M
4,84358402,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,M


In [ ]:
df = df.drop(['id'], axis='columns')

In [ ]:
#df['diagnosis'].replace(['M','B'], [1,0], inplace = True)
enc = ce.OneHotEncoder(use_cat_names=True)
df_enc = enc.fit_transform((df['diagnosis'])) 

df_enc

,diagnosis_M,diagnosis_B
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
564,1,0
565,1,0
566,1,0
567,1,0


In [ ]:

df = df.join(df_enc)
df = df.drop(['diagnosis'], axis='columns')
df

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis_M,diagnosis_B
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,1,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,1,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,1,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,1,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,1,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,1,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,1,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,1,0


In [ ]:
X = df.iloc[:, :23].values

In [ ]:
X

array([[ 17.99 ,  10.38 , 122.8  , ...,  25.38 ,  17.33 , 184.6  ],
       [ 20.57 ,  17.77 , 132.9  , ...,  24.99 ,  23.41 , 158.8  ],
       [ 19.69 ,  21.25 , 130.   , ...,  23.57 ,  25.53 , 152.5  ],
       ...,
       [ 16.6  ,  28.08 , 108.3  , ...,  18.98 ,  34.12 , 126.7  ],
       [ 20.6  ,  29.33 , 140.1  , ...,  25.74 ,  39.42 , 184.6  ],
       [  7.76 ,  24.54 ,  47.92 , ...,   9.456,  30.37 ,  59.16 ]])

In [ ]:
YData = df[['diagnosis_M','diagnosis_B']]

In [ ]:
YData = YData.values
YData

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [0, 1]])

In [ ]:

Y = df.iloc[:,23].values
X_train,X_test, Y_train, Y_test = train_test_split(X,YData, test_size=0.3, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)
print(type(X_train))
print(type(Y_train))

(398, 23)
(171, 23)
(398, 2)
(171, 2)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
scaler = StandardScaler()
scaler.fit_transform(X_train)
scaler.transform(X_test)

array([[-0.48313229, -0.13285829, -0.46029654, ..., -0.27480509,
        -0.14401878, -0.34241195],
       [ 1.34906186,  0.51103428,  1.29204314, ...,  1.78833753,
         0.1787766 ,  1.77158878],
       [ 0.36358494,  0.0762286 ,  0.38928522, ...,  0.62012735,
         0.08227076,  0.53072936],
       ...,
       [-1.20977993, -0.2611616 , -1.1400444 , ..., -1.04874434,
        -0.18561612, -1.03789852],
       [-0.10083521,  1.26659826,  0.00303674, ..., -0.05785177,
         1.23701298, -0.04127656],
       [-0.36419542, -1.19967661, -0.30118031, ..., -0.39579829,
        -1.29876099, -0.07154142]])

In [ ]:
knnCls = KNeighborsClassifier(weights='uniform',  n_neighbors=5)

In [ ]:
knnCls.fit(X_train,Y_train)

KNeighborsClassifier()

In [ ]:
Y_Pred_TEST = knnCls.predict(X_test)
print(Y_Pred_TEST)


[[0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]

In [ ]:
Y_Pred_TRAIN = knnCls.predict(X_train)
print(Y_Pred_TRAIN)

[[0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]

In [ ]:
res = knnCls.predict(X_test[50].reshape(1,-1))
res

array([[1, 0]])

In [ ]:
acc =  accuracy_score(Y_test, Y_Pred_TEST)
acc2 = accuracy_score(Y_train, Y_Pred_TRAIN)
print("TEST DATA PRedict : ", acc)
print("TRAIN DATA PRedict : ",acc2)

TEST DATA PRedict :  0.9298245614035088
TRAIN DATA PRedict :  0.9271356783919598


In [ ]:
#CLASSIFAICTION REPORT FOR Y TEST
classreport=  classification_report(Y_test, Y_Pred_TEST)
print(classreport)

              precision    recall  f1-score   support

           0       0.90      0.90      0.90        63
           1       0.94      0.94      0.94       108

   micro avg       0.93      0.93      0.93       171
   macro avg       0.92      0.92      0.92       171
weighted avg       0.93      0.93      0.93       171
 samples avg       0.93      0.93      0.93       171



In [ ]:
#CLASSIFAICTION REPORT FOR Y TRAIN
classreport=  classification_report(Y_train, Y_Pred_TRAIN)
print(classreport)

              precision    recall  f1-score   support

           0       0.95      0.85      0.90       149
           1       0.91      0.98      0.94       249

   micro avg       0.93      0.93      0.93       398
   macro avg       0.93      0.91      0.92       398
weighted avg       0.93      0.93      0.93       398
 samples avg       0.93      0.93      0.93       398



In [ ]:
# SCORE ACCURACY FOR TEST DATA
score = knnCls.score(X_test, Y_test)
score

0.9298245614035088

In [ ]:
# SCORE ACCURACY FOR TRAIN DATA
score = knnCls.score(X_train, Y_train)
score

0.9271356783919598

In [ ]:
# df['radius_mean'] = np.log(df['radius_mean'])
# df['texture_mean'] = np.log(df['texture_mean'])
# df['perimeter_mean'] = np.log(df['perimeter_mean'])
# df['area_mean'] = np.log(df['area_mean'])
# df['smoothness_mean'] = np.log(df['smoothness_mean'])
# df['compactness_mean'] = np.log(df['compactness_mean'])
# df['symmetry_mean'] = np.log(df['symmetry_mean'])
# df['fractal_dimension_mean'] = np.log(df['fractal_dimension_mean'])
# df['radius_se'] = np.log(df['radius_se'])
# df['texture_se'] = np.log(df['texture_se'])
# df['perimeter_se'] = np.log(df['perimeter_se'])
# df['smoothness_se'] = np.log(df['smoothness_se'])
# df['compactness_se'] = np.log(df['compactness_se'])
# df['symmetry_se'] = np.log(df['symmetry_se'])
# df['fractal_dimension_se'] = np.log(df['fractal_dimension_se'])
# df['radius_worst'] = np.log(df['radius_worst'])
# df['texture_worst'] = np.log(df['texture_worst'])
# df['perimeter_worst'] = np.log(df['perimeter_worst'])
# df['area_worst'] = np.log(df['area_worst'])
# df['smoothness_worst'] = np.log(df['smoothness_worst'])
# df['compactness_worst'] = np.log(df['compactness_worst'])
# df['symmetry_worst'] = np.log(df['symmetry_worst'])
# df['fractal_dimension_worst'] = np.log(df['fractal_dimension_worst'])